# Vectorizers

In this notebook, we will show how to use RedisVL to create embeddings using the built-in text embedding vectorizers. Today RedisVL supports:
1. OpenAI
2. HuggingFace
3. Vertex AI

Before running this notebook, be sure to
1. Have installed ``redisvl`` and have that environment active for this notebook.
2. Have a running Redis Stack instance with RediSearch > 2.4 active.

For example, you can run Redis Stack locally with Docker:

```bash
docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest
```

This will run Redis on port 6379 and RedisInsight at http://localhost:8001.

In [1]:
# import necessary modules
import os

## Creating Text Embeddings

This example will show how to create an embedding from 3 simple sentences with a number of different text vectorizers in RedisVL.

- "That is a happy dog"
- "That is a happy person"
- "Today is a nice day"


### OpenAI

The ``OpenAITextVectorizer`` makes it simple to use RedisVL with the embeddings models at OpenAI. For this you will need to install ``openai``. 

```bash
pip install openai
```


In [2]:
import getpass

# setup the API Key
api_key = os.environ.get("OPENAI_API_KEY") or getpass.getpass("Enter your OpenAI API key: ")

In [3]:
from redisvl.vectorize.text import OpenAITextVectorizer

# create a vectorizer
oai = OpenAITextVectorizer(
    model="text-embedding-ada-002",
    api_config={"api_key": api_key},
)

test = oai.embed("This is a test sentence.")
print("Vector dimensions: ", len(test))
test[:10]

Vector dimensions:  1536


[-0.001046799123287201,
 -0.0031105349771678448,
 0.0024228920228779316,
 -0.004480978474020958,
 -0.010343699716031551,
 0.012758520431816578,
 -0.00535263866186142,
 -0.003002384677529335,
 -0.007115328684449196,
 -0.03378167003393173]

In [4]:
# Create many embeddings at once
sentences = [
    "That is a happy dog",
    "That is a happy person",
    "Today is a sunny day"
]

embeddings = oai.embed_many(sentences)
embeddings[0][:10]

[-0.017399806529283524,
 -2.3427608653037169e-07,
 0.0014656063867732882,
 -0.02562308870255947,
 -0.019890939816832542,
 0.016027139499783516,
 -0.0036763285752385855,
 0.0008253469131886959,
 0.006609130185097456,
 -0.025165533646941185]

In [5]:
# openai also supports asyncronous requests, which we can use to speed up the vectorization process.
embeddings = await oai.aembed_many(sentences)
print("Number of Embeddings:", len(embeddings))


Number of Embeddings: 3


### Huggingface

[Huggingface](https://huggingface.co/models) is a popular NLP platform that has a number of pre-trained models you can use off the shelf. RedisVL supports using Huggingface "Sentence Transformers" to create embeddings from text. To use Huggingface, you will need to install the ``sentence-transformers`` library.

```bash
pip install sentence-transformers
```

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from redisvl.vectorize.text import HFTextVectorizer


# create a vectorizer
# choose your model from the huggingface website
hf = HFTextVectorizer(model="sentence-transformers/all-mpnet-base-v2")

# embed a sentence
test = hf.embed("This is a test sentence.")
test[:10]

In [7]:
# You can also create many embeddings at once
embeddings = hf.embed_many(sentences, as_buffer=True)


### VertexAI

[VertexAI](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) is GCP's fully-featured AI platform including a number of pretrained LLMs. RedisVL supports using VertexAI to create embeddings from these models. To use VertexAI, you will first need to install the ``google-cloud-aiplatform`` library.

```bash
pip install google-cloud-aiplatform>=1.26
```

1. Then you need to gain access to a [Google Cloud Project](https://cloud.google.com/gcp?hl=en) and provide [access to credentials](https://cloud.google.com/docs/authentication/application-default-credentials). This typically accomplished with the `GOOGLE_APPLICATION_CREDENTIALS` environment variable pointing to the path of a JSON key file downloaded from your service account on GCP.
2. Lastly, you need to find your [project ID](https://support.google.com/googleapi/answer/7014113?hl=en) and [geographic region for VertexAI](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
from redisvl.vectorize.text import VertexAITextVectorizer


# create a vectorizer
vtx = VertexAITextVectorizer(
    api_config={
        "project_id": os.environ["GCP_PROJECT_ID"],
        "location": os.environ["GCP_LOCATION"]
    }
)

# embed a sentence
test = vtx.embed("This is a test sentence.")
test[:10]

## Search with Provider Embeddings

Now that we've created our embeddings, we can use them to search for similar sentences. We will use the same 3 sentences from above and search for similar sentences.

First, we need to create the schema for our index.

Here's what the schema for the example looks like in yaml for the HuggingFace vectorizer:

```yaml
index:
    name: providers
    prefix: rvl
    storage_type: hash
    key_separator: ':'

fields:
    text:
        - name: sentence
    vector:
        - name: embedding
          dims: 768
          algorithm: flat
          distance_metric: cosine
```

In [8]:
from redisvl.index import SearchIndex

# construct a search index from the schema
index = SearchIndex.from_yaml("./schema.yaml")

# connect to local redis instance
index.connect("redis://localhost:6379")

# create the index (no data yet)
index.create(overwrite=True)

In [9]:
# use the CLI to see the created index
!rvl index listall

20:13:35 [RedisVL] INFO   Indices:
20:13:35 [RedisVL] INFO   1. providers


In [10]:
# load expects an iterable of dictionaries where
# the vector is stored as a bytes buffer

data = [{"text": t,
         "embedding": v}
        for t, v in zip(sentences, embeddings)]

index.load(data)

In [11]:
from redisvl.query import VectorQuery

# use the HuggingFace vectorizer again to create a query embedding
query_embedding = hf.embed("That is a happy cat")

query = VectorQuery(
    vector=query_embedding,
    vector_field_name="embedding",
    return_fields=["text"],
    num_results=3
)

results = index.search(query.query, query_params=query.params)
for doc in results.docs:
    print(doc.text)
    print(doc.vector_distance)

That is a happy dog
0.160862445831
That is a happy person
0.273598074913
Today is a sunny day
0.744559526443
